In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# products_sheet_id = "1FB2bbvWE2nsf49KO_DkWzbL2hm9r4G8j"
# products_list = "Tablib%20Dataset"
# url = f"https://docs.google.com/spreadsheets/d/{products_sheet_id}/gviz/tq?tqx=out:csv&sheet={products_list}"

In [2]:
# convert csv datasets into dataframes

pdt_df = pd.read_csv('full_products_list.csv')
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [3]:
# import excel file from an absolute path with pandas

xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')

In [4]:
# convert xls datasets on sheet names into dataframes

dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [5]:
# drop unwanted columns from the duplicates list dataframe

new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [6]:
# cleanup data by renaming pandas series with more meaningfull names

new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'GENERIC NAME', 'Unnamed: 7': 'BRAND NAME'})

In [7]:
# drop null values based on rows

new_dp_list = new_dp_list.dropna(axis='rows')

In [ ]:
# unique_pdts_df

In [ ]:
# check to see if dataset is now clean 

new_dp_list.tail()

In [8]:
# remove the inverse labels

new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [ ]:
# check to see whether id has numerical values
# re-check after running next cell

new_dp_list.info()

In [9]:
# convert product id into numeric 

new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [ ]:
# new_row = unique_pdts_df.iloc[14]
# new_row

In [ ]:
# new_dp_list = new_dp_list.append(new_row, ignore_index=True)

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list

In [ ]:
# check unique products dataframe

unique_pdts_df.columns

In [10]:
# drop unnecessary columns from unique lists

unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [11]:
# change column name into product id for easy manipulation

unique_pdts_df = unique_pdts_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
# unique_pdts_df = unique_pdts_df.drop(labels=14, axis=0)

In [ ]:
# unique_pdts_df.reset_index(drop=True, inplace=True)

In [ ]:
unique_pdts_df

In [12]:
# give proper indices to the dataset

new_dp_list.reset_index(drop=True, inplace=True)

In [13]:
# create a new column on which we will compare our product ids
new_dp_list['NEW ID'] = ''

# create a new column on which we will compare our generic names
new_dp_list['new_generic_name'] = ''

# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            # assign the new ids to the resulting product from unique products loop and do the same for generic name
            new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
            new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]        

<ipython-input-13-459ad8c5262c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
<ipython-input-13-459ad8c5262c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]


In [14]:
# subset the rows that were filled
filled = new_dp_list[new_dp_list['NEW ID'] != '']

# filled

In [15]:
# subset the unfilled rows
unfilled = new_dp_list[new_dp_list['NEW ID'] == '']

# unfilled

In [16]:
# re-index the dataframe because dropping out rows has messed up with its indexing

unfilled.reset_index(drop=True, inplace=True)

In [ ]:
# unique_pdts_df

In [ ]:
new_dp_list

In [ ]:
new_dp_list.head()

In [17]:
unique_dict = {k:[] for k in new_dp_list['PRODUCT ID']}

for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
            
unique_dict

# loop through the unique products dataframe using its index
# for i in range(len(unique_pdts_df)):
# #     print(unique_dict)
    
#     for key, val in unique_dict.items():
# #         if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['Generic Name'][j].lower():
#         val.append(unique_pdts_df['PRODUCT ID'][i])
# #         print(val)
       

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [],
 3819: [],
 5304: [5303],
 5239: [],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [],
 418: [],
 261: [],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [],
 5234: [],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [],
 5379: [],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [ ]:
unique_dict = {k.lower():[] for k in new_dp_list['GENERIC NAME']}

for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            unique_dict[new_dp_list['GENERIC NAME'][j].lower()].append(unique_pdts_df['PRODUCT ID'][i])
            
unique_dict

In [18]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        if len(unique_dict[new_dp_list['PRODUCT ID'][j]]) == 0:
        
            # extract string from the product
            str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
            str_b = new_dp_list['GENERIC NAME'][j].lower()

            # assign ratio to both strings
            ratio = fuzz.partial_ratio(str_b, str_a)            

            # sent conditon for comparing the series
            if ratio >= 81:
                unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
unique_dict

    #                 unique_dict[key] = str_b
    #                 unique_dict.append([str_a])
    #                 print(unique_dict)
    #                 unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
    #                 unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [4745],
 3819: [3816],
 5304: [5303],
 5239: [5044],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [4786],
 418: [4786],
 261: [4786],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [4786],
 5234: [5208],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [5337],
 5379: [5337],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        if len(unique_dict[new_dp_list['GENERIC NAME'][j].lower()]) == 0:
        
            # extract string from the product
            str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
            str_b = new_dp_list['GENERIC NAME'][j].lower()

            # assign ratio to both strings
            ratio = fuzz.partial_ratio(str_b, str_a)            

            # sent conditon for comparing the series
            if ratio >= 81:
                unique_dict[new_dp_list['GENERIC NAME'][j].lower()].append(unique_pdts_df['PRODUCT ID'][i])
unique_dict

In [ ]:
unique_dict['miconazole, clebetasol and gentamycin'].append(346)
unique_dict['diclofenac 12.5mg suppositories'].append(4334)
unique_dict['kabuuti herbal cough syrup'].append(5145)
unique_dict['diclofenac tabs'].append(4328)
unique_dict['amlodipine besilate,valsatan'].append(5303)

In [19]:
unique_dict[540].append(346)
unique_dict[5391].append(4334)
unique_dict[5389].append(4334)
unique_dict[5390].append(4334)
unique_dict[4672].append(5145)
unique_dict[3146].append(4328)
unique_dict[3819].append(5303)
unique_dict[4729].append(4945)
unique_dict[4595].append(4594)
unique_dict[3819].pop(0)
unique_dict[4595].pop(0)
unique_dict[4729].pop(0)
unique_dict[3952].pop(1)

4594

In [20]:
unique_dict

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [4328],
 5312: [830],
 5416: [4745],
 3819: [5303],
 5304: [5303],
 5239: [5044],
 4685: [4686],
 4672: [5145],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [4786],
 418: [4786],
 261: [4786],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [4786],
 5234: [5208],
 3325: [3925],
 2725: [3925],
 3952: [3352],
 4595: [4594],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [4334],
 5390: [4334],
 5389: [4334],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [4945],
 5370: [647],
 540: [346],
 4868: [4867],
 5380: [5337],
 5379: [5337],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [ ]:
unique_dict['amlodipine besilate,valsatan'].pop(0)

In [ ]:
unique_dict

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):

    # loop through the list of duplicates using its index
    for j in range(len(unfilled)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
        str_b = unfilled['GENERIC NAME'][j].lower()
        
        # assign ratio to both strings
        ratio = fuzz.partial_ratio(str_b, str_a)            
        
        # sent conditon for comparing the series
        if ratio >= 81:
            print(str_b, ':', str_a, ':', ratio)
            unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]            
#                 unique_dict[key] = str_b
#                 unique_dict.append([str_a])
#                 print(unique_dict)
#                 unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
#                 unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# unfilled

In [ ]:
# combine the dataframes into one dataframe
# combined_df = pd.concat([unfilled, filled], axis=0).sort_values('Generic Name')

In [ ]:
# re-index the dataframe
# combined_df.reset_index(drop=True, inplace=True)

In [ ]:
# combined_df

### The product dataset

In [21]:
# investigate the products on columns
pdt_df.columns

Index(['ID', 'COUNTRY OF ORIGIN', 'FORMULATION',
       'RETAIL PRICE PER SMALLEST UNIT', 'RETAIL PRICE PER SMALLEST UNIT.1',
       'PACK SIZE', 'GENERIC NAME', 'BRAND NAME', 'STRENGTH', 'eshop_id'],
      dtype='object')

In [ ]:
# new_pdt_df= pdt_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'PACK SIZE', 'RETAIL PRICE PER SMALLEST UNIT', 
#              'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id'], axis=1)

In [22]:
# investigate the product's types
pdt_df.dtypes

ID                                    int64
COUNTRY OF ORIGIN                    object
FORMULATION                          object
RETAIL PRICE PER SMALLEST UNIT      float64
RETAIL PRICE PER SMALLEST UNIT.1    float64
PACK SIZE                             int64
GENERIC NAME                         object
BRAND NAME                           object
STRENGTH                             object
eshop_id                            float64
dtype: object

In [ ]:
pdt_df.info()

In [ ]:
new_dp_list.head()

In [23]:
# change product's id for ease in comparing with other dataframes
new_pdt_df = pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [24]:
# check to find out whether the new_product_df id is identical to the duplicate list id 
merged_pdt_df = new_pdt_df['PRODUCT ID'].isin(new_dp_list['PRODUCT ID'])

In [25]:
# drop the identical ids
new_pdt_df.drop(new_pdt_df[merged_pdt_df].index, inplace=True)

In [ ]:
# checkout the new dataframe
new_pdt_df.info()

In [26]:
# test on a product from the duplicates list
new_pdt_df[new_pdt_df['PRODUCT ID'] == 3951]

,PRODUCT ID,COUNTRY OF ORIGIN,FORMULATION,RETAIL PRICE PER SMALLEST UNIT,RETAIL PRICE PER SMALLEST UNIT.1,PACK SIZE,GENERIC NAME,BRAND NAME,STRENGTH,eshop_id


In [ ]:
# merged_pdt_df[merged_pdt_df['PRODUCT ID'] == 3951]

### The batches dataframe

In [ ]:
batches_df.info()

In [28]:
batches_df[batches_df['PRODUCT ID'] == 719]

,PRODUCT ID,id,created,modified,batch_number,cost_price,initial_item_count,item_count,expiration_date,manufacturing_date,product__generic_name,product__brand_name
1678,719,2398,2021-01-09 13:32:19,2021-01-09 13:32:19,1006462,467,280,532,2023-04-30,NaN,Clopidogrel,CLOPIDOGREL 75MG UK
2022,719,2049,2020-11-22 12:00:25,2020-11-22 12:00:25,PA0830A,467,90,0,2022-12-31,NaN,Clopidogrel,CLOPIDOGREL 75MG UK
3594,719,449,2020-04-27 15:13:00,2020-11-08 01:43:19,PA1247K,450,0,0,2020-10-31,2020-04-01,Clopidogrel,CLOPIDOGREL 75MG UK


In [29]:
for key, val in unique_dict.items():
    if key in batches_df['PRODUCT ID']:
        batches_df['NEW PRODUCT ID'] = batches_df['PRODUCT ID'].map(unique_dict, na_action='ignore')
print(batches_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3982 entries, 0 to 3981
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PRODUCT ID             3982 non-null   int64 
 1   id                     3982 non-null   int64 
 2   created                3982 non-null   object
 3   modified               3982 non-null   object
 4   batch_number           3982 non-null   object
 5   cost_price             3982 non-null   int64 
 6   initial_item_count     3982 non-null   int64 
 7   item_count             3982 non-null   int64 
 8   expiration_date        3982 non-null   object
 9   manufacturing_date     1179 non-null   object
 10  product__generic_name  3982 non-null   object
 11  product__brand_name    3982 non-null   object
 12  NEW PRODUCT ID         71 non-null     object
dtypes: int64(5), object(8)
memory usage: 404.5+ KB
None


In [32]:
batches_df[batches_df['PRODUCT ID'] == 719]

,PRODUCT ID,id,created,modified,batch_number,cost_price,initial_item_count,item_count,expiration_date,manufacturing_date,product__generic_name,product__brand_name,NEW PRODUCT ID
1678,719,2398,2021-01-09 13:32:19,2021-01-09 13:32:19,1006462,467,280,532,2023-04-30,NaN,Clopidogrel,CLOPIDOGREL 75MG UK,[696]
2022,719,2049,2020-11-22 12:00:25,2020-11-22 12:00:25,PA0830A,467,90,0,2022-12-31,NaN,Clopidogrel,CLOPIDOGREL 75MG UK,[696]
3594,719,449,2020-04-27 15:13:00,2020-11-08 01:43:19,PA1247K,450,0,0,2020-10-31,2020-04-01,Clopidogrel,CLOPIDOGREL 75MG UK,[696]


In [33]:
new_batches_df = batches_df

In [35]:
for key, val in unique_dict.items():
    if key in new_batches_df['PRODUCT ID']:
        new_batches_df['PRODUCT ID'] = new_batches_df['PRODUCT ID'].replace(new_batches_df['PRODUCT ID'].map(unique_dict, na_action='ignore'))

In [ ]:
new_batches_df[new_batches_df['PRODUCT ID'] == ]

In [ ]:
batches_df[batches_df['PRODUCT ID'] == 4729]

In [ ]:
batches_df[batches_df['NEW PRODUCT ID'] != NaN ]

In [ ]:
print(batches_df['PRODUCT ID'].tolist())

In [ ]:
# cleanup data by renaming pandas series with more meaningfull names
batches_df = batches_df.rename(columns={'product__generic_name': 'Generic Name', 'product__brand_name': 'Brand Name'})

In [ ]:
new_dp_list.info()

In [ ]:
# create a new column on which we will compare our product ids
batches_df['NEW ID'] = ''

# create a new column on which we will compare our generic names
batches_df['new_generic_name'] = ''

# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        generic_unique = unique_pdts_df['GENERIC NAME'][i].lower()
        generic_batch = batches_df['Generic Name'][j].lower()
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if generic_unique in generic_batch:
            
            # assign the new ids to the resulting product from unique products loop and do the same for generic name
            batches_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
            batches_df['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]  

In [ ]:
batches_df.head()

In [ ]:
# subset the rows that were filled
filled_batches = batches_df[batches_df['NEW ID'] != '']

filled_batches

In [ ]:
# subset the rows that were not filled
unfilled_batches = batches_df[batches_df['NEW ID'] == '']

unfilled_batches

In [ ]:
unfilled_batches.reset_index(drop=True, inplace=True)

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(unfilled_batches)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i]
        str_b = unfilled_batches['Generic Name'][j]
        
        # assign ratio to both strings
        ratio = fuzz.partial_ratio(str_b, str_a)
     
        # sent conditon for comparing the series
        if ratio >= 100:
            print(str_a, ':', str_b, ':', ratio)            
            unfilled_batches['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled_batches['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(unfilled_batches)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i]
        str_b = unfilled_batches['Generic Name'][j]
        unique_brand = unique_pdts_df['BRAND NAME'][i].lower()
        batch_brand = batches_df['Brand Name'][j].lower()
        
        # assign ratio to both strings
        first_ratio = fuzz.partial_ratio(str_b, str_a)
        second_ratio = fuzz.partial_ratio(batch_brand, unique_brand)
     
        # sent conditon for comparing the series
        if first_ratio >= 80 and second_ratio >= 80:
            print(str_a, ':', str_b, ':', first_ratio, second_ratio)            
            unfilled_batches['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled_batches['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
unfilled_batches.head()

In [ ]:
unfilled_batches = unfilled_batches[unfilled_batches['NEW ID'] != '']
unfilled_batches

In [ ]:
# recipe_df.columns

In [ ]:
new_recipe_df = recipe_df.drop(['id', 'created', 'modified', 'prescription', 'dosage', 'tablets', 'quantity', 'frequency', 
                                'duration', 'route', 'price_discount', 'serving_pharmacy', 'price_controller', 'rx_price',
                               'external_price', 'requires_approval', 'approved', 'dropped', 'canceling_party', 
                                'reason_for_cancelation', 'authorized_by_tmcg', 'authorized_by_insurer', 'client_decision', 
                                'insurer_code'], axis=1)

In [ ]:
# new_recipe_df.info()

In [ ]:
# new_recipe_df.head()

In [ ]:
dp_list.head()

In [ ]:
# unique_pdts_df.info()

In [ ]:
# dp_list.columns

In [ ]:
# dp_list.info()

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list.info()

In [ ]:
# unique_pdts_df.head(49)

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.info()

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_pdt_df = new_pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
new_pdt_df.head()

In [ ]:
merged_pdt_dup = pd.merge(new_dp_list, new_pdt_df, on=['PRODUCT ID'], how='inner')

In [ ]:
# merged_pdt_dup.head(51)

In [ ]:
merged_batch_dup = pd.merge(new_dp_list, new_batches_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_batch_dup.info()

In [ ]:
new_dp_list.shape

In [ ]:
# new_recipe_df

In [ ]:
new_recipe_df = new_recipe_df.rename(columns={'product_id': 'PRODUCT ID'})

In [ ]:
new_recipe_df.head()

In [ ]:
merged_recipe_dup = pd.merge(new_dp_list, new_recipe_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_recipe_dup.info()

In [ ]:
# new_pdt_df
# new_batches_df
# new_recipe_df

# merged_pdt_dup
# merged_batch_dup
# merged_recipe_dup

In [ ]:
unique_pdts_df.head()

In [ ]:
merged_batch_dup.head()

In [ ]:
new_batch = merged_batch_dup.replace()

In [ ]:
# for prod in merged_batch_dup['Generic Name']:
#     if np.where(merged_batch_dup['Generic Name'] == unique_pdts_df['GENERIC NAME'], True, False):
#         print(prod)

In [ ]:
merged_series = merged_batch_dup['Generic Name']
unique_series = unique_pdts_df['GENERIC NAME']

In [ ]:
merged_batch_dup.columns

In [ ]:
# merged_row = merged_batch_dup.loc[['Montelukast']]
# merged_row

In [ ]:
merged_batch_dup['Generic Name'].unique()

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
unique_pdts_df.columns

In [ ]:
pdt_df.head()

In [ ]:
pdt_df['NEW ID'] = ""

In [ ]:
for i in range(len(unique_pdts_df)):
    for j in range(len(pdt_df)):
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
        if unique_pdts_df['GENERIC NAME'][i].lower() in pdt_df['GENERIC NAME'][j].lower():
            pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Montelukast']

In [ ]:
pdt_df.head()

In [ ]:
unique_pdts_df.head()

In [ ]:
pdt_df[pdt_df['NEW ID'] != ""]

In [ ]:
list(pdt_df['GENERIC NAME'].unique())

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
'Montelukast'in(pdt_df['GENERIC NAME'])

In [ ]:
new_dp_list.head(20)

In [ ]:
new_dp_list.columns

In [ ]:
new_dp_list[new_dp_list['Generic Name'] == 'Diclofenac 12.5mg Suppositories']

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Diclofenac']

In [ ]:
new_dp_list.head(10)

In [ ]:
print(new_dp_list['Generic Name'])

In [ ]:
new_dp_list.info()

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'].str.contains('Loratadine')]

In [ ]:
pattern = "+./'"

In [ ]:
unique_pdts_df.loc['Diclofenac 12.5mg Suppositories', 'GENERIC NAME']

In [ ]:
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
#         elif new_dp_list['Generic Name'][j].lower().contains(pat="+./'", regex=True) in new_dp_list['Generic Name'][j].lower():
#             new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]